# Inferência de Componentes Principais

Autores: Rafaella Ormond e Jose Jaime Martinez-Magana

Este script realiza Análise de Componentes Principais (PCA) para gerar covariáveis para o GWAS de características de tabagismo nas coortes LAGC.

ENTRADA: Arquivos binários PLINK2 `(.pgen/.psan/.pvar)` + genoma de referência `(.fa)` → controle de qualidade e conversão de formato
SAÍDA: Arquivos VCF com controle de qualidade separados por cromossomo (chr1-22), prontos para imputação nos servidores TOPMed/Michigan

### ***Requisitos***
### Baixar o Plink
Podemos baixar o Plink versão 1.9 e versão 2.0 seguindo as etapas indicadas no site oficial.<br>
Para instalar o plink2 [accesse aqui](https://www.cog-genomics.org/plink/2.0/)<br>
Para instalar o plink1.9 [accesse aqui](https://www.cog-genomics.org/plink/1.9/) <br>

### Baixar e Instalar o R

Você pode baixar e instalar o R a partir do Comprehensive R Archive Network (CRAN) para o seu sistema operacional:

- [R para Windows](https://cran.r-project.org/bin/windows/base/)
- [R para macOS](https://cran.r-project.org/bin/macosx/)
- [R para Linux](https://cran.r-project.org/bin/linux/)

Após instalar o R, você pode instalar os pacotes necessários conforme mostrado abaixo.

### Baixar pacotes do R
Instale os seguintes pacotes R (clique no nome para visitar a página de cada pacote):
- [GENESIS](https://bioconductor.org/packages/release/bioc/html/GENESIS.html)  
- [GWASTools](https://bioconductor.org/packages/release/bioc/html/GWASTools.html)  
- [SNPRelate](https://bioconductor.org/packages/release/bioc/html/SNPRelate.html)  
- [SeqArray](https://bioconductor.org/packages/release/bioc/html/SeqArray.html)  
- [parallel](https://stat.ethz.ch/R-manual/R-devel/library/parallel/doc/parallel.pdf) (built-in R package)  
- [BiocParallel](https://bioconductor.org/packages/release/bioc/html/BiocParallel.html)  
- [stringi](https://cran.r-project.org/package=stringi)


**Nota:** A maioria dos pacotes do Bioconductor pode ser instalada usando os seguintes comandos no R:

if (!requireNamespace("BiocManager", quietly = TRUE))
    install.packages("BiocManager")

BiocManager::install(c("GENESIS", "GWASTools", "SNPRelate", "SeqArray", "BiocParallel"))
install.packages("stringi")

### Etapas da Análise:
1) Poda de LD usando PLINK
2) Criação de arquivos GDS<br>
2.1. Criação do script para criar os arquivos gds<br>
2.2 Criar os arquivos gds
3) Cálculo de Componentes Principais usando PC-AiR<br>
3.1. Desenvolvimento do script para PCA e GRM com PCAiR e PCRelate<br>
3.2. Gerar os arquivos de PCA e GRM<br>
3.3. Salvar o arquivo de PCA

### 1. LD pruning usando PLINK

***Descrição:***
Este código é para realizar a poda de LD usando PLINK2 para análise de PCs
O formato plink2 usa pfiles, você também pode fazer a transformação para bfiles, mas se os arquivos estão divididos por cromossomos, com pfiles é mais rápido para fazer o merge deles

In [ ]:
## Primeiro, vamos realizar uma poda de LD com plink2
# Substitua pelo caminho e prefixo do nome do arquivo dos arquivos plink2
in="/caminho_para_seus_dados/prefixo_plink"
# Realizar primeira passagem da poda de LD
plink2 --pfile ${in}\
    --indep-pairwise 50 5 0.2 --geno 0.01 --mind 0.01 --maf 0.05\
    --out ${in}_indep_snps
plink2 --pfile ${in}\
    --extract ${in}_indep_snps.prune.in --make-bed\
    --out ${in}_forpcair

### 2. Criação dos arquivos GDS
***Descrição:***

Este script criará os arquivos GDS que serão utilizados para realizar o cálculo dos Componentes Principais na próxima etapa.

### 2.1 Criando o script para criar o arquivo gds
nome do script: **create_gds.Rscript**

Você pode baixar esse script no github [LINK AQUI](https://github.com/ormondr/Smoking_GWAS_LAGC/blob/main/English/02PCair/create_gds.Rscript)

In [ ]:
####################################################################################
# Este script utiliza um arquivo plink para criar um arquivo GDS como entrada para o PCAiR
####################################################################################
# definir parâmetros
# esta função usa a biblioteca optparse para adicionar argumentos ao script
# adicionando argumentos ao script
library(optparse) 
option_list = list(
    make_option(c("--plinkfile"), type="character", default=NULL,
                help="caminho para o diretório onde estão armazenados os arquivos plink, sem as extensões *bed/*bim/*fam", metavar="character"),
    make_option(c("--out"), type="character", default=NULL,
                help="nome do arquivo de saída GDS, use o caminho completo, exemplo /data/analysis/analysis.gds", metavar="character")
);

opt_parser = OptionParser(option_list=option_list);
opt = parse_args(opt_parser);

if (is.null(opt$plinkfile)){
    print_help(opt_parser)
    stop("Pelo menos um argumento deve ser fornecido (arquivo de entrada)", call.=FALSE)
}

###################################################################################
# carregar bibliotecas
library(GENESIS)
library(GWASTools)
library(SNPRelate)

# criar lista de arquivos
bed_f=paste0(opt$plinkfile,".bed",sep="")
bim_f=paste0(opt$plinkfile,".bim",sep="")
fam_f=paste0(opt$plinkfile,".fam",sep="")

# criar arquivo GDS de saída
snpgdsBED2GDS(bed.fn=bed_f,
              bim.fn=bim_f,
              fam.fn=fam_f,
              family=TRUE,
              out.gdsfn=opt$out)
# fim do script

### 2.2 Criando arquivo gds
***Descrição:***
Este script executará o script "create_gds.Rscript", criado no passo anterior <br>
Obs: necessário modificar os parâmetros para seus arquivos

In [ ]:
# criando arquivos GDS
# definir parâmetros
Rscript='/caminho_para_seus_dados/create_gds.Rscript'
inpath='/caminho_para_seus_dados/01plinkldpruned'
oupath='/caminho_para_seus_dados/02gdsprunned'
# criar arquivo de lista de jobs
do Rscript ${Rscript} --plinkfile=${inpath}/$cohort_name_hapmap_metalprs_allchrs --out=${oupath}/cohort_name_hapmap_metalprs_allchrs.gds

### 3. Cálculo de Componentes Principais Usando PC-AiR

**Descrição:**

A PCA é realizada usando a função `PC-AiR` do pacote **GENESIS**.

- Para aprender mais sobre PC-AiR e PC-Relate, veja esse site [GENESIS PCA Guide](https://bioconductor.org/packages/devel/bioc/vignettes/GENESIS/inst/doc/pcair.html)  
  - Veja a **Seção 3** para PC-AiR and **Seção 4** para PC-Relate.
  
> **Nota:**  
> PCA requer dados de LD prunning feitos previamente, que é realizado no primeiro passo deste pipeline.
> O arquivo contendo os 10 PCs principais será usado para todos os métodos de GWAS (ex: Regenie, GMMAT, Saige).
> O arquivo GRM é necessário apenas para GMMAT, mas recomendamos gerá-lo de qualquer forma — isso garante que você não precisará repetir este passo se decidir usar GMMAT mais tarde.




### 3.1. Desenvolvimento do script para PCA e GRM com PCAiR e PCRelate
*Script escrito em R.*
Desenvolvimento do script para PCA e GRM com PCAiR e PCRelate<br>
Este script não precisa ser modificado — os caminhos e nomes dos arquivos serão adicionados na próxima etapa.<br>
Nome do script: **create_grm.Rscript**

Você pode baixar esse script no github [LINK AQUI](https://github.com/ormondr/Smoking_GWAS_LAGC/blob/main/English/02PCair/create_grm.Rscript)


In [ ]:
####################################################################################
# Script R para criar GRM e PCs
# data: 25 de julho de 2025
# autor: Rafaella Ormond e Jose Jaime Martinez-Magana
####################################################################################
# Este script usa um arquivo gds para estimar PCs e GRM usando GENESIS
####################################################################################
# definir parâmetros
# esta função usa a biblioteca optparse para adicionar argumentos ao script
# adicionando argumentos ao script
library(optparse) 
option_list = list(
    make_option(c("--gdsfile"), type="character", default=NULL,
                help="caminho para o arquivo gds", metavar="character"),
    make_option(c("--out"), type="character", default=NULL,
                help="caminho de saída para objetos PCAiR, use um caminho de diretório completo e um nome potencial de arquivos. Exemplo: /home/user/out_pca", metavar="character")
);

opt_parser = OptionParser(option_list=option_list);
opt = parse_args(opt_parser);

if (is.null(opt$gdsfile)){
  print_help(opt_parser)
  stop("Pelo menos um argumento deve ser fornecido (arquivo de entrada)", call.=FALSE)
}

###################################################################################
# carregar bibliotecas
library(GENESIS)
library(GWASTools)
library(SNPRelate)
library(SeqArray)
library(parallel)
library(BiocParallel)
library(stringi);

# estabelecendo uma semente
set.seed(1000)
# estabelecendo núcleos
cores=detectCores()

# criar função para poda de LD
ld_prun=function(gds){
snpset=snpgdsLDpruning(gds,
                       method="corr",
                       slide.max.bp=10e6,
                       ld.threshold=sqrt(0.1),
                       maf=0.01,
                       missing.rate=0.01,
                       verbose=TRUE, num.thread=cores);
    pruned=unlist(snpset,
                  use.names=FALSE);
return(pruned)
}

# criar função para análise KING-robust
king_mat=function(gds){
samp.id=read.gdsn(index.gdsn(gds, "sample.id"))
ibd.robust=snpgdsIBDKING(gds, sample.id=samp.id, family.id=NULL, maf=0.01,missing.rate=0.01,num.thread=cores)
return(ibd.robust)
}


# criar função para PCAiR
pcair_r=function(gds_geno, pruned, KINGmat){
pcair=pcair(gds_geno, snp.include=pruned,
            kinobj=KINGmat, divobj=KINGmat)
return(pcair)
}

# criar função para PCAiR com filtro de amostra
pcair_r_fil=function(gds_geno, pruned, KINGmat, sample_list){
    pcair=pcair(gds_geno, snp.include=pruned,
                sample.include=sample_list,
                kinobj=KINGmat, divobj=KINGmat)
                return(pcair)
                }


# criar função para corresponder o ID da amostra no GDS com os IDs de amostra do filtro sampleID fornecido
filter_samples=function(gds_samples, incl_samples){
    matches=unique(grep(paste(incl_samples$SampleID,collapse="|"),
                        gds_samples, value=TRUE))
    return(matches)
}


# executar análise para pcs com relacionamentos
# abrir o objeto gds
gds=snpgdsOpen(opt$gdsfile);
# poda de LD
pruned=ld_prun(gds)
# construir matriz KING
KINGmat=king_mat(gds)
# ajustar matriz KING
KINGmat_m=KINGmat$kinship
# adicionar sampleIDs aos colnames e row names
colnames(KINGmat_m)=KINGmat$sample.id
rownames(KINGmat_m)=KINGmat$sample.id
# obter amostras no gds
gds_samples=read.gdsn(index.gdsn(gds, "sample.id"))
# fechar o objeto gds
snpgdsClose(gds)

# ler o objeto gds
gds=GdsGenotypeReader(filename=opt$gdsfile)

# criar um objeto de classe GenotypeData
gds_geno=GenotypeData(gds)

# executar PCAiR
PCair=pcair_r(gds_geno, pruned, KINGmat_m)
PCairpart=pcairPartition(kinobj = KINGmat_m, divobj = KINGmat_m)

# cálculo de PC relate 
gdsData=GenotypeBlockIterator(gds_geno, snpInclude=pruned)
PCrelate=pcrelate(gdsData,
                  pcs=PCair$vectors[,1:2],
                  training.set=PCairpart$unrels,
                  BPPARAM=BiocParallel::SerialParam())

# criando GRM
grm=pcrelateToMatrix(PCrelate)
# criando matriz esparsa
grm[grm<0.05]=0
grm_sparse=as.matrix(grm, sparse = TRUE)

# salvar arquivo
out=list()
out$PCair=PCair
out$PCairpart=PCairpart
out$KINGmat=KINGmat_m
out$pruned=pruned
out$PCrelate=PCrelate
out$grm=grm
out$grm_sparse=grm_sparse
saveRDS(file=paste0(opt$out,".rds",""),out)

### 3.2. Gerar os arquivos de PCA e GRM.
***Descrição:***

Este script executará o "create_grm.Rscript" criado no passo anterior.
Você precisará ajustar este passo de acordo com seus próprios arquivos de entrada.
A saída será um arquivo .RDS contendo tanto as informações de PCA quanto a Matriz de Relação Genética (GRM)

In [ ]:
# executar o script R para criar os arquivos GDS
# adicione o caminho do arquivo create_grm.Rscript gerado na etapa anterior
Rscript="create_grm.Rscript"
Rscript $Rscript \
--gdsfile=gdsfile_prunned \
--out=cohort_name.gds_prunned_grm_pca

### 3.3. Salvar o arquivo de PCA

***Descrição:***
Este script extrai os 10 principais componentes do arquivo .RDS gerado na etapa anterior e os salva em um arquivo separado para serem usados como covariáveis no Regenie.

In [ ]:
# Carregar o arquivo de resultado PCA/GRM
pca_grm <- readRDS("cohort_name.gds_prunned_grm_pca.rds")
# Extrair IDs das amostras (IIDs)
iids <- rownames(pca_grm$PCair$vectors)
# Extrair os 10 PCs principais
pcs <- pca_grm$PCair$vectors[,c(1:10)]
pcs <- as.data.frame(pcs)
colnames(pcs) <- paste0(rep("PC",10),rep(1:10))
# adicionar ID da amostra
pcs$SampleID=rownames(pcs)
covariate_table <- pcs
# Salvar em arquivo
write.table(covariate_table,
            file = "cohort_10pcs_forregenie.txt",
            quote = FALSE,
            row.names = FALSE,
            col.names = TRUE,
            sep = "\t")